# Demonstration for mlflow REST API

This notebook demonstrates use of the mlflow tracking REST api to retrieve results from mlflow experiments and place them into a pandas dataframe.

In [ ]:
from __future__ import print_function

In [ ]:
import pandas as pd
import numpy as np
import os
import os.path
import requests
import socket

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.tracking
import mlflow.sklearn

## set up to invoke mlflow tracking REST api

In [ ]:
# Assumes MFLOW_TRACKING_URI is set

REST_API_URL = os.environ['MLFLOW_TRACKING_URI'] + '/api/2.0/preview/mlflow'

print(REST_API_URL)

## List all experiments

In [ ]:
r = requests.get(REST_API_URL + '/experiments/list')
experiment_list = r.json()
experiment_list

## Retrieve data for specified experiment

In [ ]:
r = requests.get(REST_API_URL + '/experiments/get',
                 json={'experiment_id':'1'})
experiment_data = r.json()


In [ ]:
experiment_data['experiment']

In [ ]:
experiment_metadata = {'experiment_name':experiment_data['experiment']['name'],
                       'experiment_id' : experiment_data['experiment']['experiment_id']}

## Extract experiment results to pandas dataframe

### Retrieve desired experiment attributes

In [ ]:
df1 = pd.DataFrame([{k:experiment_data['experiment'][k] for k in ['name','experiment_id']}])
df1

### Extract run data (metrics, params and tags) 

In [ ]:
def extract_run_data(r):
    # seed dataframe row with experiment attributes
    ans = experiment_metadata.copy()
    
    # add run uuid
    ans.update({'run_uuid': r['run_uuid']})
    
    # retrieve metrics, params and tags from the run
    r1 = requests.get(REST_API_URL + '/runs/get',
                      json={'run_uuid': r['run_uuid']})
    run_data = r1.json()['run']['data']
    
    # populate dataframe row with metrics, params and tags 
    for k in run_data.keys():    
        try:
            data = {k+'_'+x['key']:x['value'] for x in run_data[k]}
        except:
            data = {}

        ans.update(data)
        
    # return the dataframe row
    return ans

In [ ]:
df = pd.DataFrame([extract_run_data(r) for r in experiment_data['runs']])
df

In [ ]:
df[['params_alpha','params_l1_ratio','params_learning_rate','params_max_depth']] = \
    df[['params_alpha','params_l1_ratio','params_learning_rate','params_max_depth']].apply(pd.to_numeric)

In [ ]:
df.dtypes

## Display experiment results sorted by metric

In [ ]:
df.sort_values('metrics_r2',ascending=False)

In [ ]:
df.shape